## Creating a Pipeline

In this exercise, you will implement a pipeline that includes a *transformer* to vectorize features of a flight, and an *estimator* to train a classification model. The resulting trained *PipelineModel* can then be used as a transformer to predict whether or not a flight will be late.

### Import Spark SQL and Spark ML Libraries

First, import the libraries you will need:

In [ ]:
import org.apache.spark.sql.types._
import org.apache.spark.sql.functions._

import org.apache.spark.ml.Pipeline
import org.apache.spark.ml.feature.VectorAssembler
import org.apache.spark.ml.classification.LogisticRegression

### Load Source Data
The data for this exercise is provided as a CSV file containing details of flights. The data includes specific characteristics (or *features*) for each flight, as well as a column indicating how many minutes late or early the flight arrived.

You will load this data into a DataFrame and display it.

In [ ]:
val csv = spark.read.option("inferSchema","true").option("header", "true").csv("wasb:///data/flights.csv")
csv.show()

### Prepare the Data
Most modeling begins with exhaustive exploration and preparation of the data. In this example, the data has been cleaned for you. You will simply select a subset of columns to use as *features* and create a Boolean *label* field named **label** with the value **1** for flights that arrived 15 minutes or more after the scheduled arrival time, or **0** if the flight was early or on-time.

In [ ]:
val data = csv.select($"DayofMonth", $"DayOfWeek", $"OriginAirportID", $"DestAirportID", $"DepDelay", ($"ArrDelay" > 15).cast("Int").alias("label"))
data.show()

### Split the Data
It is common practice when building supervised machine learning models to split the source data, using some of it to train the model and reserving some to test the trained model. In this exercise, you will use 70% of the data for training, and reserve 30% for testing. In the testing data, the **label** column is renamed to **trueLabel** so you can use it later to compare predicted labels with known actual values.

In [ ]:
val splits = data.randomSplit(Array(0.7, 0.3))
val train = splits(0)
val test = splits(1).withColumnRenamed("label", "trueLabel")
val train_rows = train.count()
val test_rows = test.count()
println("Training Rows: " + train_rows + " Testing Rows: " + test_rows)

### Define the Pipeline
A pipeline consists of a a series of *transformer* and *estimator* stages that typically prepare a DataFrame for modeling and then train a predictive model. In this case, you will create a simple pipeline with two stages:
- An transformer that uses the **VectorAssembler** class to vectorize feature columns
- An estimator that uses a **LogisticRegression** class to train a classification model.

In [ ]:
val assembler = new VectorAssembler().setInputCols(Array("DayofMonth", "DayOfWeek", "OriginAirportID", "DestAirportID", "DepDelay")).setOutputCol("features")
val lr = new LogisticRegression().setLabelCol("label").setFeaturesCol("features").setMaxIter(10).setRegParam(0.3)
val pipeline = new Pipeline().setStages(Array(assembler, lr))

### Run the Pipeline as an Estimator
The pipeline itself is an estimator, and so it has a **fit** method that you can call to run the pipeline on a specified DataFrame. In this case, you will run the pipeline on the training data to train a model. 

In [ ]:
val model = pipeline.fit(train)
println("Pipeline complete!")

### Test the Pipeline Model
The model produced by the pipeline is a transformer that will apply all of the stages in the pipeline to a specified DataFrame and apply the trained model to generate predictions. In this case, you will transform the **test** DataFrame using the pipeline to generate label predictions.

In [ ]:
val prediction = model.transform(test)
val predicted = prediction.select("features", "prediction", "probability", "trueLabel")
predicted.show(100)

The resulting DataFrame is produced by applying all of the transformations in the pipline to the test data. The **prediction** column contains the predicted value for the label, and the **trueLabel** column contains the actual known value from the testing data. It looks like there are a mix of correct and incorrect predictions - later in this course you'll learn how to measure the accuracy of a model.